# Amazon Food Reviews Analysis using Spacy & Dask

## Purpose
The purpose of this kernel is to illustrate the application of Dask and Spacy for Multiclass Text classification problem where the classes are unbalanced. 

## Methodology
Setup the Dask distributed to handle the text preprocessing and model building in parallel. The model utilizes spacy tokenizer, Hashing vectorizer for text preprocessing. For model building SGD classifier of scikit learn is then used within Incremental wrapper of Dask framework.


## Improvements
- Need to find better ways to partition the data for faster analysis;

## Results

1. Mix of Unigram + Bigram with Class Weights
    
       Class    precision    recall  f1-score   support

         1       0.58      0.69      0.63     10327
         2       0.35      0.45      0.39      6000
         3       0.39      0.43      0.41      8638
         4       0.53      0.29      0.38     16028
         5       0.85      0.88      0.86     72748
 
                    
  - accuracy       0.72


## Suggested next steps
- More extensive text clean up can be done using spacy with better exploratory analysis of text;
- Certain fields within the Dataset where ignored which can be incorporated for better accuracy;
- Hyperparameter Tuning.

# Setup

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

PATH = '/kaggle/input/amazon-fine-food-reviews/Reviews.csv'  

/kaggle/input/amazon-fine-food-reviews/database.sqlite
/kaggle/input/amazon-fine-food-reviews/hashes.txt
/kaggle/input/amazon-fine-food-reviews/Reviews.csv


In [2]:
import numpy as np
import pandas as pd
import string
import spacy
import pickle

import dask.dataframe as dd
from dask.distributed import Client
from dask_ml.model_selection import train_test_split
from dask_ml.feature_extraction.text import HashingVectorizer
from dask_ml.wrappers import Incremental
from dask_ml.metrics import accuracy_score

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report


In [3]:
from dask.distributed import Client

client = Client(threads_per_worker=2,
                n_workers=5, memory_limit='3GB')
client

Client Scheduler: tcp://127.0.0.1:45455 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 10 Memory: 15.00 GB


# Data processing

In [4]:
reqd = ['Text', 'Score']
Reviews_df = dd.read_csv(PATH,
                         usecols = reqd,
                         blocksize=20e6,
                         dtype={'Score': 'float'},
                         engine='python',
                         encoding='utf-8',
                         error_bad_lines=False)

In [5]:
Reviews_df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 2 entries, Score to Text
dtypes: object(1), float64(1)

In [6]:
Reviews_df

,Score,Text
npartitions=16,,
,float64,object
,...,...
...,...,...
,...,...
,...,...


#### Initially work with subsample of the data for rapid prototyping

In [7]:
# frac = 1.0
# Reviews_df = Reviews_df.sample(frac=frac, replace=True)

In [8]:
Reviews_df['Score'].value_counts().compute()

5.0    363122
4.0     80655
1.0     52268
3.0     42640
2.0     29769
Name: Score, dtype: int64

In [9]:
X = Reviews_df['Text']
ylabels = Reviews_df['Score'] 

In [10]:
keys = np.unique(ylabels.compute())
values = class_weight.compute_class_weight('balanced',
                                           keys,
                                           ylabels.compute())
class_weights = dict(zip(keys, values))

In [11]:
class_weights

{1.0: 2.1751511441034665,
 2.0: 3.8191004064630993,
 3.0: 2.666294559099437,
 4.0: 1.4095939495381564,
 5.0: 0.3130925694394721}

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2)

In [13]:
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

In [14]:
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

## HashingVectorizer

> This text vectorizer implementation uses the hashing trick to find the token string name to feature integer index mapping.
> This strategy has several advantages:
* > it is very low memory scalable to large datasets as there is no need to store a vocabulary dictionary in memory
* > it is fast to pickle and un-pickle as it holds no state besides the constructor parameters
* > it can be used in a streaming (partial fit) or parallel pipeline as there is no state computed during fit.

> There are also a couple of cons (vs using a CountVectorizer with an in-memory vocabulary):
* > there is no way to compute the inverse transform (from feature indices to string feature names) which can be a problem when trying to introspect which features are most important to a model.
* > there can be collisions: distinct tokens can be mapped to the same feature index. However in practice this is rarely an issue if n_features is large enough (e.g. 2 ** 18 for text classification problems).
* > no IDF weighting as this would render the transformer stateful.

In [15]:
from dask_ml.feature_extraction.text import HashingVectorizer
hw_vector = HashingVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1, 2), n_features=2**20)

In [16]:
%time
Feature_pipeline = Pipeline([('vectorizer', hw_vector)])
Pipeline_Model = Feature_pipeline.fit(X_train.values)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.87 µs


In [17]:
Text_preprocess_pipe = pickle.dumps(Pipeline_Model)

In [18]:
Pipeline_Model = pickle.loads(Text_preprocess_pipe)

In [19]:
%time
X_transformed = Pipeline_Model.transform(X_train)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


In [1]:
%%time

import joblib
estimator = SGDClassifier(random_state=10, max_iter=200, loss='modified_huber',class_weight = class_weights, n_jobs=-1)
classifier = Incremental(estimator)
Model = classifier.fit(X_transformed,
               y_train,
               classes=list(class_weights.keys()))

NameError: name 'Incremental' is not defined

In [21]:
predictions = Model.predict(Pipeline_Model.transform(X_test))
predictions

dask.array<_predict, shape=(nan,), dtype=int64, chunksize=(nan,), chunktype=numpy.ndarray>

In [22]:
accuracy_score(y_test, predictions)

0.7197859176714592

In [23]:
ML_Model = pickle.dumps(Model)

In [24]:
%time
Model = pickle.loads(ML_Model)
# X = Model.predict_proba(X_transformed).compute()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 9.06 µs


In [25]:
%time
x_test_transformed = Pipeline_Model.transform(X_test)
y_pred = Model.predict(x_test_transformed).compute()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


### Train Accuracy

In [26]:
%time
print(classification_report(y_train,
                            Model.predict(Pipeline_Model.transform(X_train)).compute()))

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.06 µs
              precision    recall  f1-score   support

         1.0       0.63      0.75      0.69     41872
         2.0       0.39      0.60      0.47     23830
         3.0       0.49      0.49      0.49     34064
         4.0       0.59      0.34      0.43     64530
         5.0       0.86      0.88      0.87    290370

    accuracy                           0.75    454666
   macro avg       0.59      0.61      0.59    454666
weighted avg       0.75      0.75      0.74    454666



### Test Accuracy

In [27]:
%time
print(classification_report(y_test, y_pred))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.54 µs
              precision    recall  f1-score   support

         1.0       0.59      0.70      0.64     10396
         2.0       0.32      0.49      0.38      5939
         3.0       0.41      0.41      0.41      8576
         4.0       0.53      0.30      0.39     16125
         5.0       0.85      0.87      0.86     72752

    accuracy                           0.72    113788
   macro avg       0.54      0.55      0.54    113788
weighted avg       0.72      0.72      0.71    113788



In [28]:
client.close()

## References
* https://examples.dask.org/machine-learning/text-vectorization.html
* https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html
* https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html